In [20]:
# pip install flask requests pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.6 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 14.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
This script demonstrates how to get an access token from the Spotify API using the client credentials flow.
"""
from config import CLIENT_ID, CLIENT_SECRET
import requests

# Define the URL and headers
url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

# Prepare the data payload
data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

# Make the POST request
response = requests.post(url, headers=headers, data=data)

# Check if the request was successful and print the response
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Access Token:", access_token)
else:
    print("Failed to get token:", response.status_code, response.text)


Access Token: BQAApqKWse3UysovlPEHI9fCQTIXsSYfe7fojsG6H7yA_TS8CV4v-EjmPcDZKZdVpHWj7D-kVg7DBYYHkWyrPDpUoCYdcSGBjh7CbdaKmLvJqPbeoYk


In [ ]:
"""
Runs the Flask server to handle Spotify authorization and token retrieval.
Uses token to fetch user's tracks and audio features and add them to a DataFrame.
"""
import subprocess
import requests
import json
import time
import pandas as pd

# Step 1: Start your Flask server using subprocess
flask_process = subprocess.Popen(['python', 'spotify-login.py'])

# Step 2: Prompt the user to authorize the app
print("Please go to http://localhost:8888/login to authorize the app.")
print("After authorizing, press Enter to continue...")
input()  # Wait for user input after authorization

# Step 3: Load the access token from the saved JSON file
def get_access_token():
    try:
        with open('spotify_token.json', 'r') as token_file:
            tokens = json.load(token_file)
            return tokens.get('access_token')
    except FileNotFoundError:
        print("Token file not found. Ensure you have completed the authorization.")
        return None

# Fetch the access token
access_token = get_access_token()
if access_token:
    print("Access token obtained successfully.")
else:
    print("Failed to obtain access token.")

# Step 4: Function to get user's saved tracks with pagination
def get_all_saved_tracks(access_token):
    url = "https://api.spotify.com/v1/me/top/tracks"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    all_tracks = []
    offset = 0
    limit = 50  # Maximum limit allowed by Spotify API

    while True:
        params = {
            'type': 'tracks',
            'time_range': 'long_term',
            'limit': limit,
            'offset': offset
        }
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        items = data.get('items', [])
        if not items:
            break
        
        # Collect track information
        for i, track in enumerate(items):
            all_tracks.append({
                'track_id': track['id'],
                'track_name': track['name'],
                'artist': track['artists'][0]['name'],
                'explicit': track['explicit'],
                'popularity': track['popularity'],
                'album_name': track['album']['name']
            })
        
        # Increment offset for next batch
        offset += limit
        print(f"Fetched {len(all_tracks)} tracks so far...")

        # Avoid hitting rate limits
        time.sleep(0.1)

    return all_tracks

# Step 5: Function to get audio features for multiple tracks
def get_audio_features(access_token, track_ids):
    url = "https://api.spotify.com/v1/audio-features"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    audio_features = []

    # Fetch in batches of 100 (Spotify API limit)
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i + 100]
        params = {
            'ids': ','.join(batch_ids)
        }
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        items = data.get('audio_features', [])
        
        # Collect track information
        for i, track in enumerate(items):
            if track is None:
                continue
            audio_features.append({
                'track_id': track['id'],
                'duration_ms': track['duration_ms'], 
                'acousticness': track['acousticness'], 
                'danceability': track['danceability'], 
                'energy': track['energy'], 
                'instrumentalness': track['instrumentalness'], 
                'key': track['key'], 
                'liveness': track['liveness'], 
                'loudness': track['loudness'], 
                'mode': track['mode'], 
                'speechiness': track['speechiness'], 
                'tempo': track['tempo'], 
                'valence': track['valence']
            })
        
        # Avoid hitting rate limits
        time.sleep(0.1)
    
    return audio_features

# Step 6: Fetch all saved tracks and audio features, then combine into a DataFrame
if access_token:
    print("Fetching all saved tracks...")
    all_tracks = get_all_saved_tracks(access_token)
    
    if all_tracks:
        # Convert track data to a DataFrame
        df_tracks = pd.DataFrame(all_tracks)
        
        # Extract track IDs
        track_ids = df_tracks['track_id'].tolist()
        
        # Fetch audio features for all tracks
        print("Fetching audio features for tracks...")
        audio_features = get_audio_features(access_token, track_ids)
        df_audio_features = pd.DataFrame(audio_features)

        # Merge track data with audio features
        df = pd.merge(df_tracks, df_audio_features, left_on='track_id', right_on='track_id', how='inner')
        df['rank'] = range(1, len(df_audio_features) + 1)
        df['explicit'] = df['explicit'].astype(int)
        
        # Reorder columns for readability
        columns_order = ['rank', 'track_name', 'artist', 'album_name', 'duration_ms', 
                         'explicit', 'popularity', 'acousticness', 'danceability', 
                         'energy', 'instrumentalness', 'key', 'liveness', 
                         'loudness', 'mode', 'speechiness', 'tempo', 'valence']
        df = df[columns_order]

        # Save the DataFrame to a JSON file
        df.to_json('tracks.json', orient='records', lines=False)

        # Display the DataFrame
        print("All Tracks with Audio Features:")
        display(df)
    else:
        print("No tracks found or failed to fetch tracks.")
else:
    print("Access token is missing. Unable to proceed.")

# Step 7: Terminate the Flask server once done
flask_process.terminate()

: 